In [35]:
import numpy as np
import pandas as pd
from datetime import datetime
import yfinance as yf
import torch
import torch.nn as nn

torch.manual_seed(42)

In [2]:
stock = "META"

end = datetime.now()
start = datetime(end.year - 1, end.month, end.day)

In [71]:
stock_data = yf.download(stock, start, end)
stock_data.astype("double")
stock_data.head(3)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-12-27,117.930000,118.599998,116.050003,116.879997,116.879997,21392300
2022-12-28,116.250000,118.150002,115.510002,115.620003,115.620003,19612500
2022-12-29,116.400002,121.029999,115.769997,120.260002,120.260002,22366200


In [73]:
stock_data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,251.000000,251.000000,251.000000,251.000000,251.000000,2.510000e+02
mean,257.425260,261.312271,254.341474,258.004223,258.004223,2.428857e+07
std,64.864148,65.286986,63.994050,64.619124,64.619124,1.312703e+07
min,116.250000,118.150002,115.510002,115.620003,115.620003,5.467500e+06
25%,204.184998,207.730003,202.680000,205.680000,205.680000,1.720465e+07
50%,284.500000,289.049988,279.399994,285.089996,285.089996,2.074290e+07
75%,310.440002,314.540009,304.869995,310.675003,310.675003,2.663555e+07
max,355.579987,357.200012,353.450012,354.829987,354.829987,1.504757e+08


In [75]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 251 entries, 2022-12-27 to 2023-12-26
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       251 non-null    float64
 1   High       251 non-null    float64
 2   Low        251 non-null    float64
 3   Close      251 non-null    float64
 4   Adj Close  251 non-null    float64
 5   Volume     251 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 13.7 KB


#### Creating train/test validation split:
An LSTM time series model looks back on previous historical time range data to predict future data to come in the same timeseries. So in our case we will have our "X" input be a sequence of historical data (e.g. Open, High, Low, Close, Adj Close, Volume over the last 60 days) and our "y" input be the price the following day/s we are trying to predict (e.g. stock price on the 61st day).

In [76]:
num_days = 60
train_size = 0.8
raw_stock_data = stock_data.values
formatted_stock_data = []

for i in range(len(raw_stock_data)-num_days):
    formatted_stock_data.append(raw_stock_data[i: i+num_days])

train = np.array(formatted_stock_data[:int(len(formatted_stock_data)*train_size)])
test = np.array(formatted_stock_data[int(len(formatted_stock_data)*train_size):])

print(f"train shape = {train.shape}")
print(f"test shape = {test.shape}")

train shape = (152, 60, 6)
test shape = (39, 60, 6)


In [77]:
x_train = torch.from_numpy(train[:,:-1,:])
y_train = torch.from_numpy(train[:,-1,:3:4])

x_test = torch.from_numpy(test[:,:-1,:])
y_test = torch.from_numpy(test[:,-1,:3:4])

print(f"x_train shape = {x_train.shape}")
print(f"y_train shape = {y_train.shape}")
print(f"x_test shape = {x_test.shape}")
print(f"y_test shape = {y_test.shape}")

x_train shape = torch.Size([152, 59, 6])
y_train shape = torch.Size([152, 1])
x_test shape = torch.Size([39, 59, 6])
y_test shape = torch.Size([39, 1])


In [78]:
input_size = 6
hidden_size = 32
num_layers = 2
batch_first = True
bias = True
dropout = 0

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, batch_first, bias, dropout):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first, bias, dropout)
        self.linear = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x


In [91]:
model = LSTM(input_size, hidden_size, num_layers, batch_first, bias, dropout)
x_train = x_train.to(torch.float32)

loss_foo = torch.nn.MSELoss()
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)